In [35]:
# PART I: BIG PICTURE
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix # optional
from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

np.random.seed(42) 


crashes1= pd.read_csv("va_crashes.csv")





#add an injuries column
def injury1(x):
    if x == 'K' or x =='A' or x == 'B' or x == 'C':
        return 1
    else:
        return 0
    
    
    
crashes1['injury'] = crashes1['Crash_Severity'].apply(injury1)


#print(crashes.loc[crashes['injury'] == 1, ['Crash_Severity']])

#Data Cleaning 
y= crashes1.injury
X_everything= crashes1.drop('injury', axis=1)




crashes = X_everything.drop(['OBJECTID','A_Crash','A_People','B_Crash','Driverinjurytype','B_People','X', 'Y', 'Crash_Dsc', 'Crash_Dt','Crash_Military_Tm','C_Crash','C_People','Crash_Severity','District_C', 'Fatal_Crashes', 'Document_Nbr', 'Pedestrians_Killed','Juris_Code', 'Vehicle_Body_Type_Cd', 'Juris_Name', 'Local_Case_Cd','Mpo_Name','Passage','Passgen','Pedgen','Passinjurytype','Pdo_Crash','Rns_Mp','Pedage','Pedinjurytype','Rte_Nm','Vehiclenumber','Vsp_1'], axis=1)

#X_train, X_test, y_train, y_test = train_test_split(crashes,y, test_size=int(len(crashes1)*0.20))

#Def drop columns that we do not want


#Features we want that are categorial variables 
coll_type = crashes[['Collision_Type']]
light_con = crashes[['Light_Condition']]
roadway = crashes[['Roadway_Surface_Cond']]
weather = crashes[['Weather_Condition']]
age = crashes[['Driverage']]
gender = crashes[['Drivergen']]
reason = crashes[['Driver_Action_Type_Cd']]
place = crashes[['Plan_Dist_Name']]


temp = crashes
temp = temp.drop('Collision_Type', axis=1)
temp = temp.drop('Light_Condition', axis=1)
temp = temp.drop('Roadway_Surface_Cond', axis=1)
temp = temp.drop('Driverage', axis=1)
temp = temp.drop('Drivergen', axis = 1)
temp = temp.drop('Driver_Action_Type_Cd', axis = 1)
temp = temp.drop('Plan_Dist_Name', axis = 1)
X = temp.drop('Weather_Condition', axis=1)


#temp2 = X_test
#temp2 = temp2.drop('Collision_Type', axis=1)
#temp2 = temp2.drop('Light_Condition', axis=1)
#temp2 = temp2.drop('Roadway_Surface_Cond', axis=1)
#temp2 = temp2.drop('Driverage', axis=1)
#temp2 = temp2.drop('Drivergen', axis = 1)
#temp2 = temp2.drop('Driver_Action_Type_Cd', axis = 1)
#temp2 = temp2.drop('Plan_Dist_Name', axis = 1)
#X2 = temp2.drop('Weather_Condition', axis=1)

 

#print(X_train.head(10))

In [36]:
crashes.head()

,Collision_Type,Crash_Year,Driverage,Drivergen,Driver_Action_Type_Cd,Gps_Latitude_Nbr,Gps_Longitude_Nbr,Light_Condition,Plan_Dist_Name,Roadway_Surface_Cond,Weather_Condition
0,1. Rear End,2016,"18,19","Male,Male","12. Following Too Close,1. No Improper Action",-79.060080,37.392420,5. Darkness - Road Not Lighted,Region 2000,2. Wet,4. Mist
1,2. Angle,2017,"32,51,33","Female,Male,Male","11. Did Not Have Right-of-Way,1. No Improper A...",-76.159620,36.837310,4. Darkness - Road Lighted,Hampton Roads,2. Wet,4. Mist
2,9. Fixed Object - Off Road,2013,16,Male,23. Driver Distraction,-77.387501,38.982137,2. Daylight,Northern Virginia,1. Dry,1. No Adverse Condition (Clear/Cloudy)
3,9. Fixed Object - Off Road,2017,49,Male,9. Other Improper Passing,-77.757740,37.692890,2. Daylight,Richmond Regional,1. Dry,1. No Adverse Condition (Clear/Cloudy)
4,1. Rear End,2014,"18,21","Male,Female","12. Following Too Close,1. No Improper Action",-76.194990,36.814570,2. Daylight,Hampton Roads,1. Dry,1. No Adverse Condition (Clear/Cloudy)


In [37]:
np.sum(crashes.isnull())


Collision_Type             0
Crash_Year                 0
Driverage                183
Drivergen                  0
Driver_Action_Type_Cd      0
Gps_Latitude_Nbr           6
Gps_Longitude_Nbr          6
Light_Condition            0
Plan_Dist_Name             0
Roadway_Surface_Cond       0
Weather_Condition          0
dtype: int64

In [38]:
##Applying the pipelienes on the data
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder



#regular pipeline for quantative data 
num_pipeline = Pipeline([
    ('imputer', Imputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])


#hot encode the categorical datas we want to keep 

#e = OneHotEncoder(sparse = False)
#h = e.fit_transform(coll_type)
#print(h)

coll_name =['Collision_Type']
light_name = ['Light_Condition']
roadway_name = ['Roadway_Surface_Cond']
weather_name = ['Weather_Condition']
#age_name = ['Driverage']
gender_name = ['Drivergen']
reason_name = ['Driver_Action_Type_Cd']
place_name = ['Plan_Dist_Name']


num_attribs = list(X) #list of numerical only values which is crash
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('collision', OneHotEncoder(), coll_name),
    ('light', OneHotEncoder(), light_name),
    ('roadway_condition', OneHotEncoder(), roadway_name),
    ('weather', OneHotEncoder(), weather_name),
    #('driver_age', OneHotEncoder(), age_name), #had NaN values so idk how to hot encode and deal with that
    ('driver_gender', OneHotEncoder(), gender_name),
    ('reason', OneHotEncoder(), reason_name),
    ('location', OneHotEncoder(), place_name),
    
    
])


num_attribs = list(X2)
test_full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('collision', OneHotEncoder(), coll_name),
    ('light', OneHotEncoder(), light_name),
    ('roadway_condition', OneHotEncoder(), roadway_name),
    ('weather', OneHotEncoder(), weather_name),
    #('driver_age', OneHotEncoder(), age_name), #had NaN values so idk how to hot encode and deal with that
    ('driver_gender', OneHotEncoder(), gender_name),
    ('reason', OneHotEncoder(), reason_name),
    ('location', OneHotEncoder(), place_name),
    
    
])


crashes_prepared = full_pipeline.fit_transform(crashes)
crashes_prepared.shape

X_train, X_test, y_train, y_test = train_test_split(crashes_prepared,y, test_size=int(len(crashes1)*0.20))

#X_train = full_pipeline.fit_transform(X_train) #cleaned scaled data
#X_test = test_full_pipeline.fit_transform(X_test)

/Users/Kajal217/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/Kajal217/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [39]:
print(X_train)

  (0, 564)	1.0
  (0, 457)	1.0
  (0, 45)	1.0
  (0, 41)	1.0
  (0, 28)	1.0
  (0, 19)	1.0
  (0, 17)	1.0
  (0, 2)	0.5296944766128389
  (0, 1)	0.5780972768609143
  (0, 0)	-0.899995284112756
  (1, 570)	1.0
  (1, 234)	1.0
  (1, 46)	1.0
  (1, 36)	1.0
  (1, 25)	1.0
  (1, 19)	1.0
  (1, 3)	1.0
  (1, 2)	-0.5677787793359059
  (1, 1)	-1.5021842107012193
  (1, 0)	-0.899995284112756
  (2, 566)	1.0
  (2, 235)	1.0
  (2, 53)	1.0
  (2, 40)	1.0
  (2, 28)	1.0
  :	:
  (8029, 19)	1.0
  (8029, 3)	1.0
  (8029, 2)	-0.2728224039587244
  (8029, 1)	0.27090128024707055
  (8029, 0)	-0.3071829583330623
  (8030, 569)	1.0
  (8030, 458)	1.0
  (8030, 59)	1.0
  (8030, 36)	1.0
  (8030, 25)	1.0
  (8030, 22)	1.0
  (8030, 10)	1.0
  (8030, 2)	-0.3155263173841328
  (8030, 1)	0.445944718462234
  (8030, 0)	1.4712540190060186
  (8031, 569)	1.0
  (8031, 234)	1.0
  (8031, 59)	1.0
  (8031, 36)	1.0
  (8031, 25)	1.0
  (8031, 19)	1.0
  (8031, 3)	1.0
  (8031, 2)	-0.3009288035249251
  (8031, 1)	0.33120581508822394
  (8031, 0)	-0.30718295833

In [40]:
#Linear Regression

from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)


crash_predictions = lin_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, crash_predictions)
lin_rmse = np.sqrt(lin_mse)
print("Linear Regression RMSE" , lin_rmse)


#This is not a bad indicator because typical injury results range from 0 to 1, there being a fatal injury, to there being no fatal injury,
#so a 0.43 means that a 




from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)
crash_predictions2 = tree_reg.predict(X_train)
tree_mse = mean_squared_error(y_train, crash_predictions2)
tree_rmse = np.sqrt(tree_mse)
print('tree' , tree_rmse) #Since we got 0 this means the model has badly overfit the data



from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train) 
crash_predictions3 = forest_reg.predict(X_train)
forest_mse = mean_squared_error(y_train, crash_predictions3)
forest_rmse = np.sqrt(forest_mse)
print('Random Forest RMSE' , forest_rmse)




Linear Regression RMSE 0.43713977601761483
tree 0.0


/Users/Kajal217/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest RMSE 0.20386272332174749


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],voting='hard')
voting_clf.fit(X_train, y_train)
print(X_train)
print('here')
print(X_test)


/Users/Kajal217/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Kajal217/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Kajal217/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


  (0, 0)	-0.8900373764177365
  (0, 1)	0.5818630276444663
  (0, 2)	0.5294818451275602
  (0, 17)	1.0
  (0, 19)	1.0
  (0, 28)	1.0
  (0, 40)	1.0
  (0, 44)	1.0
  (0, 416)	1.0
  (0, 509)	1.0
  (1, 0)	-0.8900373764177365
  (1, 1)	-1.5014668822365973
  (1, 2)	-0.5680072801020536
  (1, 3)	1.0
  (1, 19)	1.0
  (1, 25)	1.0
  (1, 35)	1.0
  (1, 45)	1.0
  (1, 215)	1.0
  (1, 515)	1.0
  (2, 0)	-1.4810898945977513
  (2, 1)	0.3625370917939431
  (2, 2)	1.0078517654981691
  (2, 3)	1.0
  (2, 19)	1.0
  :	:
  (8029, 28)	1.0
  (8029, 40)	1.0
  (8029, 94)	1.0
  (8029, 215)	1.0
  (8029, 514)	1.0
  (8030, 0)	1.4741726963023234
  (8030, 1)	0.44951681429527335
  (8030, 2)	-0.3157511706216078
  (8030, 10)	1.0
  (8030, 22)	1.0
  (8030, 25)	1.0
  (8030, 35)	1.0
  (8030, 57)	1.0
  (8030, 417)	1.0
  (8030, 514)	1.0
  (8031, 0)	-0.29898485823772153
  (8031, 1)	0.33460977375521817
  (8031, 2)	-0.3011534456847759
  (8031, 3)	1.0
  (8031, 19)	1.0
  (8031, 25)	1.0
  (8031, 35)	1.0
  (8031, 57)	1.0
  (8031, 215)	1.0
  (8031, 

In [42]:
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix,f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Training your svm here
svm_clf = LinearSVC(C=1, loss="hinge", random_state=42)

#y_train_1 = (y_train == 1)

svm_clf.fit(X_train, y_train)
#do fitting on training values 



# Testing your svm here
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_curve

#X_test =tvect.transform(X_test)
y_pred = svm_clf.predict(X_test) #predicted y scores 
#predict on test values 

accuracy = accuracy_score(y_test,y_pred)
print("accuracy: ", accuracy) # 0.0.9141304347826087

precision_s = precision_score(y_test,y_pred) #how accurate your guesses from x_test are from model for actual y_test values
print("precision: ", precision_s) #0.9405099150141643

recall_s = recall_score(y_test,y_pred) 
print("recall score:" , recall_s) #0.8512820512820513


f1 = f1_score(y_test,y_pred)
print("f1_score:", f1)  #0.8936742934051144







accuracy:  0.6661684105630294
precision:  0.6241610738255033
recall score: 0.13154172560113153
f1_score: 0.21728971962616822
